# Debugging

Every so often, you will run into a situation where you simulation fails, either gracefully with a nice error message that tells you exactly what went wrong or with a dreaded segmentation fault. When such errors occur, OpenMC gives you several tools to figure out what's going on. First, the most important thing you'll want to do when debugging is making sure that you've compiled OpenMC with debug flags on.

```sh
cmake -Ddebug=on ..
make
```

With debug flags on, when a segfault or other catastrophic error occurs, you should get a "traceback" which shows exactly which line in the source code the segfault happened at. This may give you some indication of what's wrong. If you're confused about a traceback, your best bet is to send a message to the OpenMC [user's group](groups.google.com/forum/#!forum/openmc-users) and seek help there.

## Run-time settings

There are a number of run-time settings that can also help with debugging. If you are experiencing problems with your geometry, you may want to run in geometry debugging mode, which can be enabled either by a command-line flag (if you're running directly from a command line) or through the `geometry_debug` argument of `openmc.run()`.

In [1]:
import openmc

import sys
sys.path.append('../..')
from inputs import *

In [2]:
m = openmc.Material()
m.add_nuclide('U235', 1.0)
m.set_density('g/cc', 1.0)
mats = openmc.Materials([m])
mats.export_to_xml()

s1 = openmc.Sphere(R=0.9)
s2 = openmc.Sphere(R=1.0, boundary_type='vacuum')
c1 = openmc.Cell(fill=m, region=-s1)
c2 = openmc.Cell(fill=m, region=-s2)

univ = openmc.Universe()
univ.add_cells((c1, c2))
geom = openmc.Geometry(univ)
geom.export_to_xml()

In [3]:
settings = openmc.Settings()
settings.source = openmc.Source(space=openmc.stats.Point())
settings.particles = 10
settings.batches = 1
settings.inactive = 0
settings.export_to_xml()

In [4]:
openmc.run(geometry_debug=True)


                               %%%%%%%%%%%%%%%
                          %%%%%%%%%%%%%%%%%%%%%%%%
                       %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                     %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                   %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                  %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                                   %%%%%%%%%%%%%%%%%%%%%%%%
                                    %%%%%%%%%%%%%%%%%%%%%%%%
                ###############      %%%%%%%%%%%%%%%%%%%%%%%%
               ##################     %%%%%%%%%%%%%%%%%%%%%%%
               ###################     %%%%%%%%%%%%%%%%%%%%%%%
               ####################     %%%%%%%%%%%%%%%%%%%%%%
               #####################     %%%%%%%%%%%%%%%%%%%%%
               ######################     %%%%%%%%%%%%%%%%%%%%
               #######################     %%%%%%%%%%%%%%%%%%
                #######################     %%%%%%%%%%%%%%%%%
                ######################     %%%%%%%%%%

1

OpenMC tells us that we have two cells that are overlapping.

Another useful option is to generate detailed information about the history of a single source particle. The `Settings.trace` option allows us to show all events that happen for particular particle. Particles are identified by (batch, generation, particle), so if we wanted to show event information for particle 10 in generation 1 of batch 2, we'd set `settings.trace = (2, 1, 10)`.

In [5]:
settings.trace = (1, 1, 1)
settings.export_to_xml()
openmc.run()


                               %%%%%%%%%%%%%%%
                          %%%%%%%%%%%%%%%%%%%%%%%%
                       %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                     %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                   %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                  %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                                   %%%%%%%%%%%%%%%%%%%%%%%%
                                    %%%%%%%%%%%%%%%%%%%%%%%%
                ###############      %%%%%%%%%%%%%%%%%%%%%%%%
               ##################     %%%%%%%%%%%%%%%%%%%%%%%
               ###################     %%%%%%%%%%%%%%%%%%%%%%%
               ####################     %%%%%%%%%%%%%%%%%%%%%%
               #####################     %%%%%%%%%%%%%%%%%%%%%
               ######################     %%%%%%%%%%%%%%%%%%%%
               #######################     %%%%%%%%%%%%%%%%%%
                #######################     %%%%%%%%%%%%%%%%%
                ######################     %%%%%%%%%%

1

If you want to see the event information for *every* particle, this can be enabled by setting the run verbosity to 10 (the highest level).

In [6]:
settings.verbosity = 10
settings.export_to_xml()
openmc.run()


                               %%%%%%%%%%%%%%%
                          %%%%%%%%%%%%%%%%%%%%%%%%
                       %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                     %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                   %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                  %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                                   %%%%%%%%%%%%%%%%%%%%%%%%
                                    %%%%%%%%%%%%%%%%%%%%%%%%
                ###############      %%%%%%%%%%%%%%%%%%%%%%%%
               ##################     %%%%%%%%%%%%%%%%%%%%%%%
               ###################     %%%%%%%%%%%%%%%%%%%%%%%
               ####################     %%%%%%%%%%%%%%%%%%%%%%
               #####################     %%%%%%%%%%%%%%%%%%%%%
               ######################     %%%%%%%%%%%%%%%%%%%%
               #######################     %%%%%%%%%%%%%%%%%%
                #######################     %%%%%%%%%%%%%%%%%
                ######################     %%%%%%%%%%

1

Now we can see why no fission sites were banked -- every particle in the simulation just leaked out of the geometry without having a collision.

## Restarting Simulations

The statepoint files that we have been working with to look at simulation results also have a second purpose -- they can be used to restart a simulation. This is especially useful for debugging a long simulation where you don't want to have to restart the simulation from the beginning just to figure out what went wrong.

As we've seen, by default a statepoint file is only written at the very last batch of our simulation. We can change this behavior with the `Settings.statepoint` attribute which should be a dictionary of the form `{'batches': [10, 50, 100]}`, i.e. mapping a string to an iterable of integers. For example, if you wanted to generate a statepoint at every single batch and you have `N` batches, you could set

```Python
settings.statepoint = {'batches': range(1, N + 1)}
```

To actually restart a run, use the `restart_file` argument of `openmc.run()`.

In [7]:
pincell = PinCellInputSet()
pincell.export()

In [8]:
openmc.run()


                               %%%%%%%%%%%%%%%
                          %%%%%%%%%%%%%%%%%%%%%%%%
                       %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                     %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                   %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                  %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                                   %%%%%%%%%%%%%%%%%%%%%%%%
                                    %%%%%%%%%%%%%%%%%%%%%%%%
                ###############      %%%%%%%%%%%%%%%%%%%%%%%%
               ##################     %%%%%%%%%%%%%%%%%%%%%%%
               ###################     %%%%%%%%%%%%%%%%%%%%%%%
               ####################     %%%%%%%%%%%%%%%%%%%%%%
               #####################     %%%%%%%%%%%%%%%%%%%%%
               ######################     %%%%%%%%%%%%%%%%%%%%
               #######################     %%%%%%%%%%%%%%%%%%
                #######################     %%%%%%%%%%%%%%%%%
                ######################     %%%%%%%%%%

0

Let's say we want to run another 50 batches. We just need to change `settings.batches` and then restart the run.

In [9]:
pincell.settings.batches += 50
pincell.settings.export_to_xml()

In [10]:
openmc.run(restart_file='statepoint.50.h5')


                               %%%%%%%%%%%%%%%
                          %%%%%%%%%%%%%%%%%%%%%%%%
                       %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                     %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                   %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                  %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                                   %%%%%%%%%%%%%%%%%%%%%%%%
                                    %%%%%%%%%%%%%%%%%%%%%%%%
                ###############      %%%%%%%%%%%%%%%%%%%%%%%%
               ##################     %%%%%%%%%%%%%%%%%%%%%%%
               ###################     %%%%%%%%%%%%%%%%%%%%%%%
               ####################     %%%%%%%%%%%%%%%%%%%%%%
               #####################     %%%%%%%%%%%%%%%%%%%%%
               ######################     %%%%%%%%%%%%%%%%%%%%
               #######################     %%%%%%%%%%%%%%%%%%
                #######################     %%%%%%%%%%%%%%%%%
                ######################     %%%%%%%%%%

0

For some errors that occur in OpenMC, a "particle restart file" will be generated. When that happens, there is a special particle restart mode that repeats the simulation of that particle alone (allowing you to get more verbose output in order to debug things further). To run a particle restart, simply pass the name of the particle restart file to the `restart_file` argument of `openmc.run()`.